In [27]:
import pandas as pd

df = pd.read_csv('data-2025-01.csv')
df.head()

,game_id,white_name,black_name,winner,white_elo,black_elo,opening
0,1,EL_DON_DE_LOS_ELOTES,incik1352,2,1657,1610,Bishop's Opening: Berlin Defense
1,2,Cammyd8173,SeanIrons,2,971,1093,Queen's Gambit Declined: Albin Countergambit
2,3,CHOQUEDF,jundumo,2,1693,1698,Mikenas Defense
3,4,Rafly168,anonerer,1,1759,1704,"French Defense: Tarrasch Variation, Guimard De..."
4,5,Vaquangnam47,robtsr,2,1137,1184,French Defense: Queen's Knight


In [40]:
# Gruppierung nach Opening und winner
summary = (
    df.groupby("opening")["winner"]
    .value_counts()
    .unstack(fill_value=0)
    .rename(columns={0: "draw", 1: "white_wins", 2: "black_wins"})
)

# Gesamtanzahl der Spiele pro Opening ergänzen
summary["total_games"] = summary.sum(axis=1)

# Optional sortieren nach meistgespieltem Opening
summary = summary.sort_values(by="total_games", ascending=False)

summary["win_rate_white"] = (summary.white_wins / summary.total_games).round(2)
summary["win_rate_black"] = (summary.black_wins / summary.total_games).round(2)

summary.reset_index(inplace=True)

summary.head()

winner,opening,draw,white_wins,black_wins,total_games,win_rate_white,win_rate_black
0,Queen's Pawn Game: Accelerated London System,11,74,53,138,0.54,0.38
1,Philidor Defense,5,53,57,115,0.46,0.50
2,Queen's Pawn Game,3,48,46,97,0.49,0.47
3,Four Knights Game: Italian Variation,7,37,39,83,0.45,0.47
4,Pirc Defense,5,43,29,77,0.56,0.38


## Vergleich von Winrates

In [41]:
def detect_opening_side(opening):
    if pd.isna(opening):
        return "unknown"
    o = opening.lower()
    if "defense" in o:
        return "black"
    elif "opening" in o or "gambit" in o:
        return "white"
    else:
        return "unknown"

summary["opening_played_by"] = summary["opening"].apply(detect_opening_side)
summary.head()

winner,opening,draw,white_wins,black_wins,total_games,win_rate_white,win_rate_black,opening_played_by
0,Queen's Pawn Game: Accelerated London System,11,74,53,138,0.54,0.38,unknown
1,Philidor Defense,5,53,57,115,0.46,0.50,black
2,Queen's Pawn Game,3,48,46,97,0.49,0.47,unknown
3,Four Knights Game: Italian Variation,7,37,39,83,0.45,0.47,unknown
4,Pirc Defense,5,43,29,77,0.56,0.38,black


In [43]:
def elo_group(elo):
    if elo < 1200:
        return "< 1200 (Anfänger)"
    elif elo <= 2000:
        return "1200–2000 (Fortgeschritten)"
    else:
        return "> 2000 (Meister)"

df["avg_elo"] = (df["white_elo"] + df["black_elo"]) / 2
df["elo_group"] = df["avg_elo"].apply(elo_group)

counts = df.groupby('elo_group')['winner'].value_counts(normalize=True).unstack(fill_value=0)

counts = counts.rename(columns={
    0: "Remis",
    1: "Weiß gewinnt",
    2: "Schwarz gewinnt"
})

counts = counts.round(2)

elo_order = ["< 1200 (Anfänger)", "1200–2000 (Fortgeschritten)", "> 2000 (Meister)"]
counts = counts.loc[elo_order]

counts.head()

winner,Remis,Weiß gewinnt,Schwarz gewinnt
elo_group,,,
< 1200 (Anfänger),0.05,0.52,0.44
1200–2000 (Fortgeschritten),0.05,0.48,0.46
> 2000 (Meister),0.11,0.46,0.42
